In [5]:
!pip install ipyleaflet

831.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [10]:
from collections import namedtuple
from math import radians, sin, cos, atan2, sqrt, degrees

# Point is a convenience wrapper to hold the co-ordinates belonging to
# a particular "point" - i.e. the base station location, or the remote
# device location.
#
# Units: lat, long = radians. alt = meters.
Point = namedtuple('Point', ['lat', 'lon', 'alt'])

def point(lat: float, lon: float, alt: float) -> Point:
  return Point(lat=radians(lat), lon=radians(lon), alt=alt)

# Direction is a convenience wrapper that holds the information about the
# "direction" between two Points; these should translate pretty well for
# use with a 2-axis gimbal.
#
# Units: bearing = degrees. elevation, distance = meters.
Direction = namedtuple('Direction', ['bearing', 'elevation', 'distance'])

class Location:
  def __init__(self, location: Point, precision: int = 2) -> None:
    self.location = location
    self.precision = precision

  def update(self, **kwargs):
    update = { k : kwargs[k] for k in kwargs if k in ['lat', 'lon', 'alt'] }
    if not (len(update) > 0 and len(update) < 3):
      raise ValueError(f"location update must consist of a subset of ['lat', 'lon', 'alt'] - got {list(kwargs.keys())}")

    self.location = self.location._replace(**update)

  def direction(self, remote: Point) -> Direction:
    """
    Accepts two "Point" tuples - one containing the current location, and one
    containing the "target" location (i.e. location of remote antenna) - and
    uses Haversine, as well as some basic geometry, to calculate the Direction.
    @todo - There's gonna be a wayyyy simpler way of doing the below; i.e. computing
          the distance with the bearing, and then reducing the angle_of_elevation
          function to atan2(height/distance).
    """

    def compass_bearing() -> tuple[float, float]:
      # @source - https://gist.github.com/jeromer/2005586 (@jeromer)
      diffLong = remote.lon - self.location.lon

      initial_bearing = atan2((sin(diffLong) * cos(remote.lat)), (
        cos(self.location.lat) * sin(remote.lat) - (sin(self.location.lat) * cos(remote.lat) * cos(diffLong))
      ))

      return (degrees(initial_bearing) + 360) % 360

    def angle_of_elevation() -> float:
      diffLong = remote.lon - self.location.lon
      diffLat  = remote.lat - self.location.lat
    
      a = sin(diffLong / 2)**2 + cos(self.location.lat) * cos(remote.lat) * sin(diffLat / 2)**2
      distance_meters = 6373000.0 * (2 * atan2(sqrt(a), sqrt(1 - a)))
      return (distance_meters, degrees(atan2(remote.alt - self.location.alt, distance_meters)))

    distance, elevation = angle_of_elevation()
    return Direction(
      bearing=round(compass_bearing(), self.precision),
      elevation=round(elevation, self.precision),
      distance=round(distance, self.precision)
    )

In [44]:
from ipyleaflet import Map, Marker, WidgetControl, AwesomeIcon
import ipywidgets as widgets

altitude_input = widgets.IntSlider(
    value=0,
    min=0,
    max=150,
    step=5,
    description='Altitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

calc_output = widgets.HTML()

DEFAULT_BASE_COORDS   = (51.5048226, -0.113632836)
DEFAULT_REMOTE_COORDS = (51.5041468, -0.118133430)

base_marker = Marker(location=DEFAULT_BASE_COORDS, icon=AwesomeIcon(
    name='compass',
    marker_color='red',
    icon_color='white',
), draggable=False)

remote_marker = Marker(location=DEFAULT_REMOTE_COORDS, icon=AwesomeIcon(
    name='arrows',
    marker_color='green',
    icon_color='darkgreen',
    spin=True
))

m = Map(center=base_marker.location, controls=[], dragging=False, zoom=17)
m.add(base_marker)
m.add(remote_marker)
m.add_control(WidgetControl(widget=altitude_input, position='topright'))
m.add_control(WidgetControl(widget=calc_output, position='bottomright'))

def handle_updated_parameters(*args, **kwargs):
    def direction_html(dir):
        out = " - ".join([
            f"<b>Distance (m)</b>: {dir.distance}", f"<b>Elevation</b>: {dir.elevation}&deg;", f"<b>Bearing</b>: {dir.bearing}&deg;"
        ])
        return f"&nbsp;{out}&nbsp;"
    
    base_location = Location(point(base_marker.location[0], base_marker.location[1], 0), 2)
    remote_point = point(remote_marker.location[0], remote_marker.location[1], altitude_input.value)
    calc_output.value = direction_html(base_location.direction(remote_point))

altitude_input.observe(handle_updated_parameters, 'value')
remote_marker.observe(handle_updated_parameters, 'location')
handle_updated_parameters()

display(m)

Map(center=[51.5048226, -0.113632836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [50]:
def geographic_tests():
  TestCase = namedtuple("TestCase", ["RemotePoint", "ExpectedDirection"])
  current_location = Location(point(51.5069574, -0.112639096, 0), 2)

  for idx, t in enumerate([
    # (Remote Point, Expected Direction)
    TestCase(point(51.4986765, -0.104676284, 0), Direction(149.09, 0, 1055.08)),
    TestCase(point(51.4987206, -0.112233761, 0), Direction(178.25, 0, 572.08)),
    TestCase(point(51.5008267, -0.119832024, 0), Direction(216.14, 0, 905.69)),
    TestCase(point(51.5020523, -0.124047118, 0), Direction(235.37, 0, 1313.57)),
    TestCase(point(51.5044345, -0.123437039, 0), Direction(249.43, 0, 1213.69)),
    TestCase(point(51.5072307, -0.121800917, 0), Direction(272.75, 0, 1019.24)),
    TestCase(point(51.5099059, -0.118168171, 0), Direction(310.59, 0, 647.99)),
    TestCase(point(51.5111140, -0.108711939, 0), Direction(30.46,  0, 523.08)),        
  ]):
    actual = current_location.direction(t.RemotePoint)

    if not compare(t.ExpectedDirection, actual):
      print(f"[geographic - {idx}] TEST FAILED. expected '{t.ExpectedDirection}', actual '{actual}'")
      return
  print("all tests passed")

def elevation_tests():
  TestCase = namedtuple("TestCase", ["BaseAltitude", "RemoteAltitude", "ExpectedElevation"])

  for idx, t in enumerate([TestCase(-5, 10, 0.81), TestCase(0, 430, 22.17)]):
    remote = point(51.4986765, -0.104676284, t.RemoteAltitude)
    actual = Location(point(51.5069574, -0.112639096, t.BaseAltitude), 2).direction(remote)

    expected = Direction(149.09, t.ExpectedElevation, 1055.08)
    if not compare(expected, actual):
      print(f"[elevation - {idx}] TEST FAILED. expected '{expected}', actual '{actual}'")
      return
  print("all tests passed")

def compare(expected: Direction, actual: Direction):
  """

  """
  return all([
      getattr(expected, prop) == getattr(actual, prop)
      for prop in ["bearing", "elevation", "distance"]
  ])

geographic_tests()
elevation_tests()

all tests passed
all tests passed
